# Computational Experiment 5: N-Body Simulations

In this experiment, you will run N-body simulations of a variety of systems to see how they evolve.

Before you begin, **make sure to read through the full notebook and understand what each function does**. Pay special attention to anything that has a **FIXME** note, which you will need to edit.

In this notebook, we will be making movies but saving them as external .mp4 files instead of embedding them in the notebook.

## 0. Import packages

In [ ]:
from gravhopper import Simulation, IC
from astropy import units as u, constants as const
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import pynbody.analysis.profile as profile

import galpy.util
galpy.util.__config__.set('astropy', 'astropy-units', 'True')
from galpy import potential, df

You will need to download [GravHopper](https://github.com/jbailinua/gravhopper).

## 1. Earth-Sun System

We will begin with a simple $N=2$ system so you can see how the package works: the Earth orbiting the Sun. First we need to set up the initial conditions (ICs) of the simulation: the Sun will be at the origin, and the Earth will be 1 au away on a circular orbit.

In [ ]:
# Sun at origin with zero velocity and mass 1 Msun
Sun = {'pos':np.array([0, 0, 0])*u.AU,
       'vel':np.array([0, 0, 0])*u.km/u.s,
       'mass':np.array([1])*u.Msun}
# Earth is at x=1 au, v_y=29.8 km/s and mass 1 Mearth
Earth = {'pos':np.array([1, 0, 0])*u.AU,
         'vel':np.array([0, 29.8, 0])*u.km/u.s,
         'mass':np.array([1])*u.Mearth}

To set up a simulation, we need to give two important parameters:
1. Time step $dt$, which defines how long between simulation steps.
2. The softening length $\epsilon$, which ensures that the force doesn't diverge when particles get too close. This is important when the particles represent samples of a smooth density field, as we'll get to later, but in this case each particle really represents a distinct object, so we don't want any softening and should set this to a very small number (for numerical reasons it's best if this isn't literally zero).

In [ ]:
# Set up a Simulation with a time step of 1/20th of a year, a very small softening
sim = Simulation(dt=0.05*u.yr, eps=0.001*u.au)

And then we add the two bodies to the ICs.

In [ ]:
sim.add_IC(Sun)
sim.add_IC(Earth)

Finally we run the simulation for 21 time steps. Each step is 1/20th of a year, so this will give us one full Earth orbit.

In [ ]:
sim.run(21)

After running, you can access the full position and velocity history using ``sim.positions`` and ``sim.velocities``. Each of these is a (Nsnap, Np, 3) array: the first dimension refers to the snapshot number, the second dimension refers to the particle number, and the third dimension refers to the coordinate (0=x, 1=y, 2=z). You can put specific numbers in for each of these, or use ``:`` to get them all.

For example, if you want to find the y velocities of all particles at snapshot number 5:
```
sim.velocities[5,:,1]
```
or if you wanted to know the 3D position of particle number 0 at the end (remember that you can use ``-1`` to get the last element):
```
sim.positions[-1,0,:]
```
or if you wanted to know the x positions of particle number 25 at all snapshots:
```
sim.positions[:,25,0]
```

So the following code will plot the x-y positions of each body over all snapshots.

In [ ]:
# Make a plot that has an aspect ratio of 1, i.e. it will be square if the axes have the same size.
plt.subplot(111, aspect=1.0)
# Plot x vs. y of particle 0 (Sun) in units of AU (default is kpc, which will look very strange!)
plt.plot(sim.positions[:,0,0].to(u.au), sim.positions[:,0,1].to(u.au), 'o', color='yellow')
# Plot x vs. y of particle 1 (Earth) as above
plt.plot(sim.positions[:,1,0].to(u.au), sim.positions[:,1,1].to(u.au), color='blue')
plt.xlabel('x (au)')
plt.ylabel('y (au)')

Look at it! Does it look like you'd expect?

Let's check out the Sun's velocity in one dimension, say $x$. You can access the time of each simulation snapshot using ``sim.times``.

In [ ]:
# Plot Sun's reflex motion vs. time (in years)
plt.plot(sim.times.to(u.yr), sim.velocities[:,0,0].to(u.m/u.s))
plt.xlabel('t (yr)')
plt.ylabel('$v_x$ (m/s)')

What do you think about this?

**FIXME:** Try changing the mass of the Earth. You will need to recreate the Simulation object each time -- there is a ``sim.reset()`` function, which will wipe out the run but it will leave the ICs and any external forces (see later), so if you want to completely change the ICs it's easiest to just create a new Simulation object by going back to the ``sim = Simulation(...`` line.

The first exoplanet-detecting spectrographs had a spectral resolution equivalent to 1 m/s. How massive does the Earth-like planet need to be to be detectable with those instruments around a nearby star, i.e. to cause a reflex motion of 1 m/s? Do you get the same answer from analytic arguments?

**FIXME**: Reset the mass of the Earth, but change the timestep ``dt`` and run the simulation for a few orbits (you will need to adjust the number of steps to be appropriate). What happens when ``dt`` is small? What happens when ``dt`` is large? How large can you make it before it starts messing up the orbit? Why?

## 2. Effects of orbital eccentricity

Let's try to make this a bit more realistic. You can use [this list of planetary data](https://nssdc.gsfc.nasa.gov/planetary/factsheet/) to look up the masses and orbital properties of some important Solar System bodies.

It's relatively easy to set up a planet at either aphelion or perihelion using this data: the "Orbital Velocity" and "Distance from Sun" rows are both averages, and conservation of angular momentum tells you that $v \propto 1/r$ throughout the orbit. For example, instead of the Earth shown above, this line will set up the Earth to be at perihelion along the x axis:

In [ ]:
#Make this the appropriate value in km/s but do not put the astropy unit in -- that will be done below.
v_Earth_at_peri = 29.8 * 149.6/147.1

Earth = {'pos':np.array([147.1e6, 0, 0])*u.km,
         'vel':np.array([0, v_Earth_at_peri, 0])*u.km/u.s,
         'mass':np.array([1])*u.Mearth}

Let's do the simulation again and see if accounting for orbital eccentricity makes a difference.

In [ ]:
# Set up a Simulation with a time step of 1/20th of a year, a very small softening
sim_e = Simulation(dt=0.05*u.yr, eps=0.001*u.au)
sim_e.add_IC(Sun)
sim_e.add_IC(Earth)
sim_e.run(21)

In [ ]:
# Make a plot that has an aspect ratio of 1, i.e. it will be square if the axes have the same size.
plt.subplot(111, aspect=1.0)
# Plot x vs. y of particle 0 (Sun) in units of AU
plt.plot(sim.positions[:,0,0].to(u.au), sim.positions[:,0,1].to(u.au), 'o', color='yellow')
# Plot x vs. y of circular Earth
plt.plot(sim.positions[:,1,0].to(u.au), sim.positions[:,1,1].to(u.au), color='blue', label='Circular')
# Plot x vs. y of eccentric Earth
plt.plot(sim_e.positions[:,1,0].to(u.au), sim_e.positions[:,1,1].to(u.au), color='darkcyan', label='Eccentric')
plt.xlabel('x (au)')
plt.ylabel('y (au)')
plt.legend()

In [ ]:
# Plot Sun's reflex motion vs. time (in years)
plt.plot(sim.times.to(u.yr), sim.velocities[:,0,0].to(u.m/u.s), color='blue', label='Circular')
plt.plot(sim_e.times.to(u.yr), sim_e.velocities[:,0,0].to(u.m/u.s), color='darkcyan', label='Eccentric')
plt.xlabel('t (yr)')
plt.ylabel('$v_x$ (m/s)')
plt.legend()

What do you think about these plots?

**FIXME:** Try changing the orbital eccentricity of the Earth, keeping everything else the same. How eccentric does it need to be to make a measurable difference? How do radial velocity curves for stars with eccentric planets differ?

## 3. Add more planets

**FIXME:** Using the orbital data from the NSSDC website, create a simulation that has Venus, Earth, Jupiter, and Saturn. Make sure to choose an appropriate value for both ``dt`` that is small enough to accurately integrate the fastest planet, and enough time steps to do at least one full orbit of the slowest planet. Set some of them at perihelion and some at aphelion. Also, don't line them all up on the same axis -- put at least a couple of them at different points in the orbit.

In [ ]:
Venus = #FIXME
Earth = #FIXME
Jupiter = #FIXME
Saturn = #FIXME

In [ ]:
sim_SS = Simulation(#FIXME
sim_SS.add_IC(#FIXME
#...more add_IC lines...
sim_SS.run(#FIXME

Plot the planet orbits.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, aspect=1.0)
ax.plot(sim_SS.positions[:,0,0].to(u.au), sim_SS.positions[:,0,1].to(u.au), 'o', color='yellow')
for i in range(1,5):
    ax.plot(sim_SS.positions[:,i,0].to(u.au), sim_SS.positions[:,i,1].to(u.au))

What do you think?

We can make a movie of the simulation using the ``movie_particles()`` function. It will write the output to an .mp4 file (you need to give it a filename). The other most useful parameters are the unit of the x and y axes, and the x and y limits of the plots that make up the movie. If you want to do a more complicated movie, you can follow the examples later on or look at the GravHopper source code of this function.

In [ ]:
# The title bar will show a timeline, but by default it is in Myr, which isn't very useful for this case.
# So first let's change the sim_SS.times array to be in units of years.
sim_SS.times = sim_SS.times.to(u.yr)
# Then make the movie, with axes in units of AU, going out to 12 AU in each direction, and points that are not too small.
sim_SS.movie_particles('SS.mp4', unit=u.au, xlim=[-12,12], ylim=[-12,12], s=8)

What do you think of this movie?

Now let's look at the Solar reflex motion again.

In [ ]:
# Plot Sun's reflex motion vs. time (in years)
plt.plot(sim_SS.times.to(u.yr), sim_SS.velocities[:,0,0].to(u.m/u.s))
plt.xlabel('t (yr)')
plt.ylabel('$v_x$ (m/s)')

**FIXME:** Use ``plt.xlim()`` and ``plt.ylim`` to zoom in on diffent parts of the plot (or you can run this all with ``%matplotlib notebook`` so you can zoom in interactively). Can you identify oscillations caused by each of the planets?

## 4. Do something interesting

Alter one of the above simulations in some interesting way to explore solar system-type dynamics. For example, you could:
 - Add one or more moons.
 - Make planets massive enough and/or nearby enough to each other that they interact with each other.
 - Throw a planet at the Sun.
 - Simulate a real exoplanet system.
 - Set up an unstable 3-body system.

## 5. Plummer sphere

Now let's set up a system with lots of particles! The simplest system we can imagine is the ergodic Plummer sphere we derived in class.

In [ ]:
# Create ICs for a Plummer sphere with characteristic scale 1pc, mass 1e6 Msun, sampled with 5000 particles
Plummer_IC = IC.Plummer(b=1*u.pc, totmass=1e6*u.Msun, N=5000)

In [ ]:
# Create a simulation with timesteps of 0.005 Myr and softening of 0.05 pc
sim_P = Simulation(dt=0.005*u.Myr, eps=0.05*u.pc)
# Add the Plummer ICs
sim_P.add_IC(Plummer_IC)

In [ ]:
# Run for 500 time steps
sim_P.run(500)

We can plot the particle distribution projected along different axes using the ``plot_particles()`` method. We'll start by looking at the initial particle positions (``snap='IC'``):

In [ ]:
fig = plt.figure(figsize=(8,9))
ax1 = fig.add_subplot(221, aspect=1.0)
ax2 = fig.add_subplot(222, aspect=1.0)
ax3 = fig.add_subplot(223, aspect=1.0)

# Plot the x-y projection of positions onto axis ax1 in units of pc with limits +/-5
sim_P.plot_particles(snap='IC', parm='pos', coords='xy',  unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax1)
# ...and the other projections
sim_P.plot_particles(snap='IC', parm='pos', coords='zy', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax2)
sim_P.plot_particles(snap='IC', parm='pos', coords='xz', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax3)

We can compare this to the final snapshot by specifying ``snap='final'``. Other options are a specific snapshot number.

In [ ]:
fig = plt.figure(figsize=(8,9))
ax1 = fig.add_subplot(221, aspect=1.0)
ax2 = fig.add_subplot(222, aspect=1.0)
ax3 = fig.add_subplot(223, aspect=1.0)

sim_P.plot_particles(snap='final', parm='pos', coords='xy',  unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax1)
sim_P.plot_particles(snap='final', parm='pos', coords='zy', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax2)
sim_P.plot_particles(snap='final', parm='pos', coords='xz', unit=u.pc, xlim=[-5,5], ylim=[-5,5], ax=ax3)

Look at these! Does the distribution appear to be in equilibrium?

We can examine other aspects of the simulation too, such as the velocities.
**FIXME:** Use ``parm='vel'`` to compare the initial and final velocity ellipsoid (note: you should probably eliminate or change the ``xlim`` and ``ylim`` parameters. What do you think?

Let's make a movie of the positions and velocities! This will save the movie to ``Plummer_posvel.mp4``; you can change this in the ``anim.save()`` line as you make other movies.

In [ ]:
fig = plt.figure(figsize=(15,7))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121, aspect=1.0)
    ax2 = fig.add_subplot(122, aspect=1.0)
    
    # First plot: particle positions xy in units of pc
    posplot = sim_P.plot_particles(snap=frame, parm='pos', coords='xy', unit=u.pc, \
                                 xlim=[-5,5], ylim=[-5,5], ax=ax1)
    # Second plot: velocity ellipsoid xy in units of km/s
    velplot = sim_P.plot_particles(snap=frame, parm='vel', coords='xy', unit=u.km/u.s, \
                                xlim=[-75,75], ylim=[-75,75], ax=ax2)
    return posplot, velplot

ms_per_frame = 40   # 25 frames per second
anim = FuncAnimation(fig, animate, frames=sim_P.Nsnap, interval=ms_per_frame)
anim.save('Plummer_posvel.mp4')

plt.close(fig)

What do you think of them? What do you think about how long it takes to do the simulation vs. to create the movie?

We can also turn a GravHopper snapshot into a pynbody snapshot using the ``pyn_snap()`` function, so we can use pynbody's analysis routines to create profiles. So let's make a movie that shows the density profile and velocity dispersion profile over the course of the simulation.

In [ ]:
fig = plt.figure(figsize=(20,8))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # Create a pynbody snapshot of the given snapshot
    snap = sim_P.pyn_snap(frame)
    # The conversion routine forces position units to kpc, but pc is much more convenient
    # for this situation, so convert it back before making the profile.
    snap['pos'].convert_units('pc')
    # Make a 3D profile of the snapshot
    prof = profile.Profile(snap, ndim=3, nbins=200)
    
    # First plot: density profile
    densplot = ax1.plot(prof['rbins'], prof['density'])
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlim(1, 50)
    ax1.set_ylim(1e-4,3e4)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())
    ax1.set_xlabel(radial_label)
    ax1.set_ylabel('$\\rho$ (${0}$)'.format(prof['density'].units.latex()))
    ax1.set_title('{0:.2f}'.format(sim_P.times[frame]))
    
    # Second plot: radial velocity dispersion profile
    sigmaplot = ax2.plot(prof['rbins'], prof['vr_disp'].in_units('km s^-1'))
    ax2.set_xscale('log')
    ax2.set_xlim(1, 50)
    ax2.set_ylim(0, 25)
    ax2.set_xlabel(radial_label)
    ax2.set_ylabel('$\\sigma_r$ (km/s)')
    
    return densplot, sigmaplot

anim = FuncAnimation(fig, animate, frames=sim_P.Nsnap, interval=ms_per_frame)
anim.save('Plummer_profiles.mp4')

plt.close(fig)

What do you think of these? How does the $\sigma_r$ plot relate to the particle velocity plots earlier? Do you expect the simulation to be in equilibrium? Does it appear to be based on these various plots and movies? How do you think I decided what value of ``dt`` to use for the simulation?

## 6. Play with the Plummer sphere

**FIXME:** Try changing the scale length ``b`` of the Plummer sphere (you will probably need to play with the limits of the various axes). How would you expect the velocity dispersion to scale with $b$ based on analytic arguments? How does that compare to what you see?

**FIXME:** Try changing the total mass of the Plummer sphere. How would you expect the velocity dispersion to scale with mass? How does that compare to what you see?

## 7. Truncated Singular Isothermal Sphere

We investigated the Truncated Singular Isothermal Sphere analytically in Assignment 4 -- now let's examine it numerically!

In [ ]:
# Create a TSIS with a mass of 1e11 Msun, a cutoff radius of 100 kpc, sampled with 5000 particles
TSIS_IC = IC.TSIS(totmass=1e11*u.Msun, maxrad=100*u.kpc, N=5000)

**FIXME:** Create a simulation with a time step of 5 Myr, softening of 0.2 kpc. Add the TSIS ICs, and run it for 500 time steps.

In [ ]:
sim_TSIS = #FIXME
sim_TSIS.add_IC(#FIXME
sim_TSIS.run(#FIXME

Let's make movies of the positions/velocities and the density/velocity dispersion profiles for this case too.

In [ ]:
fig = plt.figure(figsize=(10,5))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121, aspect=1.0)
    ax2 = fig.add_subplot(122, aspect=1.0)
    
    # First plot: particle positions xy in units of pc
    posplot = sim_TSIS.plot_particles(snap=frame, parm='pos', coords='xy', unit=u.kpc, \
                                 xlim=[-200,200], ylim=[-200,200], ax=ax1)
    # Second plot: velocity ellipsoid xy in units of km/s
    velplot = sim_TSIS.plot_particles(snap=frame, parm='vel', coords='xy', unit=u.km/u.s, \
                                xlim=[-200,200], ylim=[-200,200], ax=ax2)
    return posplot, velplot

ms_per_frame = 40   # 25 frames per second
anim = FuncAnimation(fig, animate, frames=sim_TSIS.Nsnap, interval=ms_per_frame)
anim.save('TSIS_posvel.mp4')

plt.close(fig)

In [ ]:
fig = plt.figure(figsize=(20,8))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    # Create a pynbody snapshot of the given snapshot and make a 3D profile from it
    snap = sim_TSIS.pyn_snap(frame)
    # Make a 3D profile of the snapshot
    prof = profile.Profile(snap, ndim=3, nbins=200)
    
    # First plot: density profile
    densplot = ax1.plot(prof['rbins'], prof['density'])
    ax1.set_xscale('log')
    ax1.set_yscale('log')
    ax1.set_xlim(0.2, 400)
    ax1.set_ylim(1e-1,1e10)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())
    ax1.set_xlabel(radial_label)
    ax1.set_ylabel('$\\rho$ (${0}$)'.format(prof['density'].units.latex()))
    ax1.set_title('{0:.2f}'.format(sim_TSIS.times[frame]))
    
    # Second plot: radial velocity dispersion profile
    sigmaplot = ax2.plot(prof['rbins'], prof['vr_disp'].in_units('km s^-1'))
    ax2.set_xscale('log')
    ax2.set_xlim(0.2, 400)
    ax2.set_ylim(0, 60)
    ax2.set_xlabel(radial_label)
    ax2.set_ylabel('$\\sigma_r$ (km/s)')
    
    return densplot, sigmaplot

anim = FuncAnimation(fig, animate, frames=sim_TSIS.Nsnap, interval=ms_per_frame)
anim.save('TSIS_profiles.mp4')

plt.close(fig)

How does this situation differ from the Plummer sphere? What happens at small radius? What happens near the truncation radius? What happens at large radius? Think about the TSIS question from Assignment 4 -- what would you have expected to happen based on your analysis? How does that relate to what you see here? Think about Liouville's Theorem -- how is what you see in the ``TSIS_posvel.mp4`` movie an example of that?

## 8. Do something interesting

Modify something in one of the spherical examples. For example:
 - Try changing ``dt`` significantly. What effect does it have?
 - Try changing ``N`` significantly. How does it affect what you see? How does it affect how long it takes the simulation to run?
 - Try running the simulation for much longer. Do you see any distinct long-term evolution of the system that's different from what happens on shorter timescales?
 - Use the Hernquist sphere (you can run ``help(IC.Hernquist)`` to find the parameters it takes). How does it behave compared to the other examples?
 - Use one of galpy's other spherical distribution functions (see [here](https://docs.galpy.org/en/latest/reference/df.html)) and ``IC.from_galpy_df()`` to do another IC. Note that the galpy DFs don't play nicely with astropy units, so you will need to convert the units manually.

## 9. Disk stability

We will create an exponential disk that is isothermal in the vertical direction, in a background NFW potential, and let it run. Solving for an equilibrium disk configuration is very hard, and so ``IC.expdisk`` does an approximate solution.

First we need to define the analytic NFW potential. This is both used in the simulation, so all particles will feel this extra force in addition to the force from all the other particles, and is also required by the IC code so that it knows what rotation velocities to give the particles.

In [ ]:
# Galpy NFW potential with a scale mass of 2e11 Msun and scale radius of 20 kpc
NFWpot = potential.NFWPotential(amp=2e11*u.Msun, a=20*u.kpc)

In [ ]:
# Create an exponential disk with central surface brightness of 100 Msun/pc^2, exponential scale length of 2kpc
# and vertical height of 0.2 kpc, a radial velocity dispersion scale of 20 km/s, and
# using the NFWpot rotation curve as an external force with 5000 particles
expdisk_IC = IC.expdisk(N=5000, sigma0=100*u.Msun/u.pc**2, Rd=2*u.kpc, z0=0.2*u.kpc, sigmaR_Rd=20*u.km/u.s,\
                       external_rotcurve=NFWpot.vcirc)

In [ ]:
# Create a simulation with a 2.5 Myr time step and 0.1 kpc softening
sim_disk = Simulation(dt=2.5*u.Myr, eps=0.1*u.kpc)
# Add the disk IC
sim_disk.add_IC(expdisk_IC)
# Add the NFW potential as an external force
sim_disk.add_external_force(NFWpot)

In [ ]:
# Run for 400 time steps
sim_disk.run(400)

Let's make a movie of the particle positions and velocities, both face-on and edge-on.

In [ ]:
fig = plt.figure(figsize=(15,15))

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax_xy = fig.add_subplot(221, aspect=1.0)
    ax_xz = fig.add_subplot(223, aspect=1.0)
    ax_vxy = fig.add_subplot(222, aspect=1.0)
    ax_vxz = fig.add_subplot(224, aspect=1.0)
    
    # First plot: particle positions xy in units of kpc
    plot_xy = sim_disk.plot_particles(snap=frame, parm='pos', coords='xy', unit=u.kpc, \
                                 xlim=[-20,20], ylim=[-20,20], ax=ax_xy)
    # Second plot: velocity in xy in units of km/s
    plot_vxy = sim_disk.plot_particles(snap=frame, parm='vel', coords='xy', unit=u.km/u.s, \
                                xlim=[-200,200], ylim=[-200,200], ax=ax_vxy)

    # Same but xz
    plot_xz = sim_disk.plot_particles(snap=frame, parm='pos', coords='xz', unit=u.kpc, \
                                 xlim=[-20,20], ylim=[-20,20], ax=ax_xz)
    plot_vxz = sim_disk.plot_particles(snap=frame, parm='vel', coords='xz', unit=u.km/u.s, \
                                xlim=[-200,200], ylim=[-200,200], ax=ax_vxz)
        
    return plot_xy, plot_vxy, plot_xz, plot_vxz

ms_per_frame = 40   # 25 frames per second
anim = FuncAnimation(fig, animate, frames=sim_disk.Nsnap, interval=ms_per_frame)
anim.save('disk_posvel.mp4')

plt.close(fig)

Think about what you're seeing, both initially and after it settles down. Does it reach a reasonably steady state, or does it continue to evolve?

Let's look at the evolution of the surface density and vertical density profiles.

In [ ]:
fig = plt.figure(figsize=(18,9))

plotz_max = 2.

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax_surfdens = fig.add_subplot(121)
    ax_vertdens = fig.add_subplot(122)
    
    # Create a pynbody snapshot of the given snapshot and make a 3D profile from it
    snap = sim_disk.pyn_snap(frame)
    # Make a 2D profile of the snapshot (so we get surface density as a function of cylindrical radius)
    prof = profile.Profile(snap, ndim=3, nbins=200)
    # Vertical profile that gives us density as a function of height. Use all radii between 0 and 25 kpc.
    zprof = profile.VerticalProfile(snap, 0., 25., plotz_max)
    
    # Surface density profile
    densplot = ax_surfdens.plot(prof['rbins'], prof['density'])
    ax_surfdens.set_xscale('log')
    ax_surfdens.set_yscale('log')
    ax_surfdens.set_xlim(0.1, 20)
    ax_surfdens.set_ylim(1e2,1e8)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())
    ax_surfdens.set_xlabel(radial_label)
    ax_surfdens.set_ylabel('$\\Sigma$ (${0}$)'.format(prof['density'].units.latex()))
    ax_surfdens.set_title('{0:.1f}'.format(sim_disk.times[frame]))
    
    # Vertical density profile
    vertplot = ax_vertdens.plot(zprof['rbins'], zprof['density'].in_units('Msol pc^-3'))
    ax_vertdens.set_yscale('log')
    ax_vertdens.set_xlim(0,plotz_max)
    ax_vertdens.set_ylim(3e-6, 3e-3)
    vertical_axis_label = 'z (${0}$)'.format(zprof['rbins'].units.latex())
    ax_vertdens.set_xlabel(vertical_axis_label)
    ax_vertdens.set_ylabel('$\\rho$ [M$_{\odot}$ pc$^{-3}$]')
        
    return densplot, vertplot

anim = FuncAnimation(fig, animate, frames=sim_disk.Nsnap, interval=ms_per_frame)
anim.save('disk_profiles.mp4')

plt.close(fig)

Look at these. How do they connect to how we tried to set up the disk? How do they connect to what you see in the particle movies?

We can also use profiles to make movies of the 3 velocity components. We'll plot both the mean velocities and the dispersions.

In [ ]:
fig = plt.figure(figsize=(18,18))

radius_range = [0,20]
vrange = [-15,40]

# Function that updates each frame
def animate(frame):
    fig.clf()
    ax_vr = fig.add_subplot(221)
    ax_vphi = fig.add_subplot(222)
    ax_vz = fig.add_subplot(223)
    
    # Create a pynbody snapshot of the given snapshot and make a 3D profile from it
    snap = sim_disk.pyn_snap(frame)
    # Make a 2D profile of the snapshot
    prof = profile.Profile(snap, ndim=3, nbins=200)
    radial_label = 'r (${0}$)'.format(prof['rbins'].units.latex())

    vrplot = ax_vr.plot(prof['rbins'], prof['vr'], label='$<v_R>$')
    ax_vr.plot(prof['rbins'], prof['vr_disp'], label='$\sigma_R$')
    ax_vr.plot(radius_range, [0,0], color='gray')
    ax_vr.set_xlim(*radius_range)
    ax_vr.set_ylim(*vrange)
    ax_vr.set_xlabel(radial_label)
    ax_vr.set_ylabel('Radial Velocity')
    ax_vr.legend(loc='upper right')

    vphiplot = ax_vphi.plot(prof['rbins'], prof['vphi'], label='$<v_{\phi}>$')
    ax_vphi.plot(prof['rbins'], prof['vphi_disp'], label='$\sigma_{\phi}$')
    ax_vphi.set_xlim(*radius_range)
    ax_vphi.set_ylim(0,100)  # These will be higher
    ax_vphi.set_xlabel(radial_label)
    ax_vphi.set_ylabel('Azimuthal Velocity')
    ax_vphi.legend(loc='upper left')
    
    vzplot = ax_vz.plot(prof['rbins'], prof['vz'], label='$<v_z>$')
    ax_vz.plot(prof['rbins'], prof['vz_disp'], label='$\sigma_z$')
    ax_vz.plot(radius_range, [0,0], color='gray')
    ax_vz.set_xlim(*radius_range)
    ax_vz.set_ylim(*vrange)
    ax_vz.set_xlabel(radial_label)
    ax_vz.set_ylabel('Vertical Velocity')
    ax_vz.legend(loc='upper right')
    
    return vrplot, vphiplot, vzplot

anim = FuncAnimation(fig, animate, frames=sim_disk.Nsnap, interval=ms_per_frame)
anim.save('disk_vprofs.mp4')

plt.close(fig)

Think about what you're seeing, both in terms of the mean velocities and their dispersions. How does this relate to how you see the densities evolve?

## 10. Play with the disk

**FIXME:** Take each of the disk parameters and try changing it to see what effect it has. Do the equilibrium configurations (if they exist) reflect these changes in the way you would expect? Do the presence or absence of disk instabilities change in the way you would expect?

## 11. Find a particle and trace its orbit

Let's find a particle with particular properties and trace its orbit. We'll choose a particle based on its original position and velocity. Pick a position and velocity for a particle that you would expect to be on a mostly (but not perfectly) circular orbit, i.e. one that should have a slight orbital eccentricity in the orbital plane and a slight vertical bobbing.

**FIXME:** Put your preferred particle position and velocity here.

In [ ]:
R_desired = #FIXME
z_desired = #FIXME
vR_desired = #FIXME
vz_desired = #FIXME

In [ ]:
# Compare all particles to this, relative to a "typical" distance and velocity scale
Rscale = 2*u.kpc
zscale = 0.2*u.kpc
vscale = 2*u.km/u.s
disk_R = np.sqrt(sim_disk.positions[0,:,0]**2 + sim_disk.positions[0,:,1]**2)
disk_z = sim_disk.positions[0,:,2]
disk_vR = (sim_disk.velocities[0,:,0]*sim_disk.positions[0,:,0] + sim_disk.velocities[0,:,1]*sim_disk.positions[0,:,1])/disk_R
disk_vz = sim_disk.velocities[0,:,2]
how_dissimilar = ((disk_R - R_desired)/Rscale)**2 + ((disk_z - z_desired)/zscale)**2 + \
    ((disk_vR - vR_desired)/vscale)**2 + ((disk_vz - vz_desired)/vscale)**2

In [ ]:
# np.argmin tells you which element in an array has the smallest value
best_particle = np.argmin(how_dissimilar)

**FIXME:** Check that it did an okay job using this. If not, go back and adjust your desired parameters. You won't find a perfect match, but make sure that it is "mostly but not perfectly circular".

In [ ]:
print('R: ',disk_R[best_particle])
print('z: ',disk_z[best_particle])
print('vR: ',disk_vR[best_particle])
print('vz: ',disk_vz[best_particle])

Now let's plot this particle's orbit.

**FIXME:** Plot the xy and xz trajectories of ``best_particle``.

In [ ]:
# xy and xz projections
fig = plt.figure(figsize=(10,4))
ax_xy = fig.add_subplot(121, aspect=1.0)
ax_xz = fig.add_subplot(122, aspect=1.0)
ax_xy.plot(#FIXME
ax_xz.plot(#FIXME
# Label appropriately
ax_xy.set_#FIXME...
ax_xz.set_#FIXME...

And let's plot how $R$ and $z$ evolve with time.

Note: You might want to use the ``%matplotlib notebook`` pragma to do the analysis of this.

In [ ]:
fig = plt.figure(figsize=(12,4))
ax_R = fig.add_subplot(121)
ax_z = fig.add_subplot(122)
# Define R of best_particle for convenience
best_R = np.sqrt(sim_disk.positions[:,best_particle,0]**2 + sim_disk.positions[:,best_particle,1]**2)
# time label
timelabel = 't ({0})'.format(sim_disk.times.unit)

ax_R.plot(sim_disk.times, best_R)
ax_R.set_xlabel(timelabel)
ax_R.set_ylabel('R ({0})'.format(best_R.unit))

ax_z.plot(sim_disk.times, sim_disk.positions[:,best_particle,2])
ax_z.set_xlabel(timelabel)
ax_z.set_ylabel('z ({0})'.format(sim_disk.positions.unit))

How do these orbits compare to the orbits we computed in Experiment 3? What is your best estimate for the epicyclic frequency, based on this plot? How about the vertical frequency? Are they constant? Is there any relation between the vertical frequency and $R$? What would you expect the frequencies to be at early times based on the NFW + disk potential with the surface density profile we gave and the radius you chose? Do they agree with what you measure from the plot?

## 12. Do something interesting

You know the drill... try something else! For example, you could:
 - Take some parameters to ridiculous extremes and see whether it reacts how you expect.
 - Add an N-body bulge (e.g. as a Hernquist IC).
     - If you do this, make sure to include its effect on the rotation curve when you set up the disk.
 - Use a different potential for the halo.
 - Add a time-dependent potential like a bar (note: if you do this, you will need to use the ``add_external_timedependent_force`` instead of ``add_external_force``).
 - Run for a very long time to see the long term evolution of the disk.
 - Try using a halo potential that is different from the rotation curve used to set up the disk.